<a href="https://colab.research.google.com/github/alifamarta/UTS-BigData/blob/main/BigDataPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
ls

restoran.csv  sample_data/


In [3]:
pwd

'/content'

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [7]:
# Downloading and preprocessing restoran.csv Data downloaded origianlly from https://perso.telecom-paristech.fr/eagan/class/igr204/datasets
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/restoran.csv

--2024-11-03 07:20:30--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/restoran.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/restoran.csv [following]
--2024-11-03 07:20:30--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/restoran.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-03 07:20:31 ERROR 404: Not Found.



In [8]:
!ls

restoran.csv  sample_data  spark-3.5.1-bin-hadoop3  spark-3.5.1-bin-hadoop3.tgz


In [9]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df = spark.read.csv('restoran.csv', header=True, sep=";")
df.show(5)

+----------+-----------+---------+---+----------+---------+
|    Minggu|Burger King|Sushi Tei|KFC|McDonald's|Pizza Hut|
+----------+-----------+---------+---+----------+---------+
|27/10/2019|         17|        8| 34|        31|       22|
|03/11/2019|         18|        9| 42|        35|       44|
|10/11/2019|         24|        8| 50|        34|       41|
|17/11/2019|         13|        8| 36|        31|       26|
|24/11/2019|         16|        7| 44|        33|       26|
+----------+-----------+---------+---+----------+---------+
only showing top 5 rows



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df.show(5, truncate=False)

+----------+-----------+---------+---+----------+---------+
|Minggu    |Burger King|Sushi Tei|KFC|McDonald's|Pizza Hut|
+----------+-----------+---------+---+----------+---------+
|27/10/2019|17         |8        |34 |31        |22       |
|03/11/2019|18         |9        |42 |35        |44       |
|10/11/2019|24         |8        |50 |34        |41       |
|17/11/2019|13         |8        |36 |31        |26       |
|24/11/2019|16         |7        |44 |33        |26       |
+----------+-----------+---------+---+----------+---------+
only showing top 5 rows



In [12]:
df.limit(5)

Minggu,Burger King,Sushi Tei,KFC,McDonald's,Pizza Hut
27/10/2019,17,8,34,31,22
03/11/2019,18,9,42,35,44
10/11/2019,24,8,50,34,41
17/11/2019,13,8,36,31,26
24/11/2019,16,7,44,33,26


In [13]:
df.columns

['Minggu', 'Burger King', 'Sushi Tei', 'KFC', "McDonald's", 'Pizza Hut']

In [14]:
df.printSchema()

root
 |-- Minggu: string (nullable = true)
 |-- Burger King: string (nullable = true)
 |-- Sushi Tei: string (nullable = true)
 |-- KFC: string (nullable = true)
 |-- McDonald's: string (nullable = true)
 |-- Pizza Hut: string (nullable = true)



In [15]:
df = spark.read.csv('restoran.csv', header=True, sep=";", inferSchema=True)
df.printSchema()

root
 |-- Minggu: string (nullable = true)
 |-- Burger King: integer (nullable = true)
 |-- Sushi Tei: integer (nullable = true)
 |-- KFC: integer (nullable = true)
 |-- McDonald's: integer (nullable = true)
 |-- Pizza Hut: integer (nullable = true)



In [16]:
from pyspark.sql.types import *
df.columns

['Minggu', 'Burger King', 'Sushi Tei', 'KFC', "McDonald's", 'Pizza Hut']

In [17]:
# Creating a list of the schema in the format column_name, data_type
labels = [
     ('Minggu',StringType()),
     ('Burger King',IntegerType()),
     ('Sushi Tei',IntegerType()),
     ('KFC',IntegerType()),
     ("McDonald's",IntegerType()),
     ('Pizza Hut',IntegerType())
]

In [18]:
# Creating the schema that will be passed when reading the csv
schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

StructType([StructField('Minggu', StringType(), True), StructField('Burger King', IntegerType(), True), StructField('Sushi Tei', IntegerType(), True), StructField('KFC', IntegerType(), True), StructField('McDonald's', IntegerType(), True), StructField('Pizza Hut', IntegerType(), True)])

In [19]:
df = spark.read.csv('restoran.csv', header=True, sep=";", schema=schema)
df.printSchema()
# The schema comes as we gave!

root
 |-- Minggu: string (nullable = true)
 |-- Burger King: integer (nullable = true)
 |-- Sushi Tei: integer (nullable = true)
 |-- KFC: integer (nullable = true)
 |-- McDonald's: integer (nullable = true)
 |-- Pizza Hut: integer (nullable = true)



In [20]:
df.show(truncate=False)

+----------+-----------+---------+---+----------+---------+
|Minggu    |Burger King|Sushi Tei|KFC|McDonald's|Pizza Hut|
+----------+-----------+---------+---+----------+---------+
|27/10/2019|17         |8        |34 |31        |22       |
|03/11/2019|18         |9        |42 |35        |44       |
|10/11/2019|24         |8        |50 |34        |41       |
|17/11/2019|13         |8        |36 |31        |26       |
|24/11/2019|16         |7        |44 |33        |26       |
|01/12/2019|17         |10       |34 |28        |27       |
|08/12/2019|15         |9        |38 |29        |29       |
|15/12/2019|16         |9        |42 |33        |31       |
|22/12/2019|18         |12       |47 |36        |34       |
|29/12/2019|16         |10       |52 |37        |39       |
|05/01/2020|13         |8        |33 |25        |26       |
|12/01/2020|14         |7        |37 |25        |24       |
|19/01/2020|14         |8        |48 |29        |25       |
|26/01/2020|15         |8        |42 |29

In [27]:
# analisa pencarian restoran yang dicari lebih dari 10
from pyspark.sql.functions import col

df = spark.read.csv('restoran.csv', header=True, sep=";", inferSchema=True)
res1 = df.filter((col('KFC')>10))
res2 = df.filter((col('Pizza Hut')>10))
res3 = df.filter((col('Burger King')>10))
res4 = df.filter((col('Sushi Tei')>10))
res5 = df.filter((col("McDonald's")>10))
print("KFC searches that are over 10: "+str(res1.count()))
print("Pizza Hut searches that are over 10: "+str(res2.count()))
print("Burger King searches that are over 10: "+str(res3.count()))
print("Sushi Tei searches that are over 10: "+str(res4.count()))
print("McDonald's searches that are over 10: "+str(res5.count()))


KFC searches that are over 10: 262
Pizza Hut searches that are over 10: 233
Burger King searches that are over 10: 47
Sushi Tei searches that are over 10: 11
McDonald's searches that are over 10: 262
